# Marine biogeochemistry assessment

Production date: YYYY-MM-DD

**Please note that this repository is used for development and review, so quality assessments should be considered work in progress until they are merged into the main branch.**

Produced by: C3S2_521 contract.

## 🌍 Use case: Use case listed here in full 

## ❓ Quality assessment question
* **In most cases there should be one question listed here in bold**
* **(In some cases a second related/follow-up question may be included)**

**‘Context paragraph’ (no title/heading)** - a very short introduction before the assessment statement describing approach taken to answer the user question. One or two key references could be useful, if the assessment summarises literature.

**Background**
Check documentation: Only one model? Mean?

## 📢 Quality assessment statement

```{admonition} These are the key outcomes of this assessment
:class: note
* Finding 1
* Finding 2
* Finding 3
* etc
```

## 📋 Methodology

A ‘free text’ introduction to the data analysis steps or a description of the literature synthesis, with a justification of the approach taken, and limitations mentioned. **Mention which CDS catalogue entry is used, including a link, and also any other entries used for the assessment**.
[Marine biogeochemistry data for the Northwest European Shelf and Mediterranean Sea from 2006 up to 2100 derived from climate projections](https://doi.org/10.24381/cds.dcc9295c)

**Note:** This notebook is currently just a brain-dump in anticipation of starting the actual quality assessment at a later stage.

E.g. 'The analysis and results are organised in the following steps, which are detailed in the sections below:' 

**[](section-setup)**
 * Sub-steps or key points listed in bullet below. No strict requirement to match and link to sub-headings.

**[](section-analysis)**
 * Sub-steps or key points listed in bullet below. No strict requirement to match and link to sub-headings.

**[](section-results)**
 * Sub-steps or key points listed in bullet below. No strict requirement to match and link to sub-headings.

Any further notes on the method could go here (explanations, caveats or limitations).

## 📈 Analysis and results

(section-setup)=
### 1. Code setup

#### Imports

In [ ]:
# Input / Output
from pathlib import Path
import earthkit.data as ekd
import warnings

# General data handling
import numpy as np
np.seterr(divide="ignore")  # Ignore divide-by-zero warnings
import pandas as pd
import xarray as xr
from functools import partial
from dask.array.core import PerformanceWarning
warnings.simplefilter(action="ignore", category=PerformanceWarning)

# Visualisation
import earthkit.plots as ekp
from earthkit.plots.styles import Style
import matplotlib.pyplot as plt
plt.rcParams["grid.linestyle"] = "--"
from tqdm import tqdm  # Progress bars

# Visualisation in Jupyter book -- automatically ignored otherwise
try:
    from myst_nb import glue
except ImportError:
    glue = None

#### Helper functions

##### General

In [ ]:
# Type hints for helper functions
from typing import Callable, Optional, Iterable

# For pre-defining functions
from functools import partial

##### Downloading data

In [ ]:
## Data downloading
def domain_to_request(domain: ekp.geo.domains.Domain) -> dict:
    """ From an earthkit-plots domain, generate a request for earthkit-data / cdsapi. """
    bbox = domain.bbox.to_latlon_bbox()

    # Round
    north = int(np.ceil(bbox.north) + 1)
    south = int(np.floor(bbox.south) - 1)
    west = int(np.floor(bbox.west) - 1)
    east = int(np.ceil(bbox.east) + 1)
    
    area = [north, west, south, east]
    return {"area": area}

##### Data (pre-)processing

In [ ]:
## Loops for convenience
def loop_over_(*args, progress=True, **kwargs) -> tqdm:
    """ Generate a tqdm progressbar; inverts `disable` keyword """
    return tqdm(*args, disable=not progress, leave=False, **kwargs)

def loop_over_ensemble_members(data: xr.Dataset, **tqdm_kwargs) -> tqdm:
    """ Loop over ensemble members in `data`, with a progress bar. """
    return loop_over_(data.groupby("number"), unit="member", **tqdm_kwargs)

def loop_over_data_variables(data: xr.Dataset, **tqdm_kwargs) -> tqdm:
    """ Loop over variable keys in `data`, with a progress bar. """
    return loop_over_(data.data_vars.keys(), unit="variable", **tqdm_kwargs)

##### Visualisation

In [ ]:
def _add_textbox_to_subplots(text: str, *axs: Iterable[plt.Axes | ekp.Subplot], right=False) -> None:
    """ Add a text box to each of the specified subplots. """
    # Get the plt.Axes for each ekp.Subplot
    axs = [subplot.ax if isinstance(subplot, ekp.Subplot) else subplot for subplot in axs]

    # Set up location
    x = 0.95 if right else 0.05
    horizontalalignment = "right" if right else "left"

    # Add the text
    for ax in axs:
        ax.text(x, 0.95, text, transform=ax.transAxes,
        horizontalalignment=horizontalalignment, verticalalignment="top",
        bbox={"facecolor": "white", "edgecolor": "black", "boxstyle": "round",
              "alpha": 1})

In [ ]:
def decorate_fig(fig: ekp.Figure, *, title: Optional[str]="") -> None:
    """ Decorate an earthkit figure with land, coastlines, etc. """
    # Add progress bar because individual steps can be very slow for large plots
    with tqdm(total=5, desc="Decorating", leave=False) as progressbar:
        fig.land()
        progressbar.update()
        fig.coastlines()
        progressbar.update()
        fig.borders()
        progressbar.update()
        fig.gridlines(linestyle=plt.rcParams["grid.linestyle"])
        progressbar.update()
        fig.title(title)
        progressbar.update()

(section-analysis)=
### 2. Analysis

#### WFDE5

In [ ]:
# domain = ekp.geo.domains.Domain.from_string("Europe")
domain = ekp.geo.domains.Domain.from_string("Italy")  # Smaller domain for testing
year = 2040
months = list(range(1, 13))

In [ ]:
request_domain = domain_to_request(domain)

request_time = {
    "year": [f"{year}"],
    "month": [f"{month:02}" for month in months],
}

In [ ]:
request_smp_base = {
    "origin": "nemo_ersem",
    "vertical_resolution": "water_column",
    "time_aggregation": "day",
    "experiment": ["rcp8_5"],
    "variable": ["total_chlorophyll_a"],
}

In [ ]:
# Download WFDE5
SMP_ID = "sis-marine-properties"

request_smp = request_smp_base | request_time

data_smp = ekd.from_source("cds", SMP_ID, request_smp).to_xarray()
data_smp = data_smp.chunk({"depth": -1})

In [ ]:
data_smp.sel(time="2040-08-31", method="nearest")

In [ ]:
data_smp

In [ ]:
# Show variables
date = "2040-08-31"
surface = 0.5
data = data_smp.sel(time=date, depth=surface, method="nearest")
n_var = len(data.data_vars)

In [ ]:
data["chl"].max().values

In [ ]:
# Show variables
date = "2040-08-31"
surface = 0.5
data = data_smp.sel(time=date, depth=surface, method="nearest")
n_var = len(data.data_vars)
fig = ekp.Figure(rows=n_var, columns=1, size=(4, 4*n_var))

# Plot single variable
subplot = fig.add_map(domain="Europe")
subplot.grid_cells(data, z="chl")#, style=styles[var])

# Legend at the bottom
for subplot in fig.subplots:
    try:
        subplot.legend(location="right")
    except ValueError:
        continue

# Decorate figures
decorate_fig(fig)

# Show result
plt.show()
plt.close()

#### ERA5

(section-results)=
### 5. Results

#### Results Subsections
Describe what is done in this step/section and what the `code` in the cell does (if code is included). 

If this is the **results section**, we expect the final plots to be created here with a description of how to interpret them, and what information can be extracted for the specific use case and user question. The information in the 'quality assessment statement' should be derived here. 

In [ ]:
# collapsable code cell

# code is included for transparency but also learning purposes and gives users the chance to adapt the code used for the assessment as they wish

## ℹ️ If you want to know more

### Key resources

List some key resources related to this assessment. E.g. CDS entries, applications, dataset documentation, external pages.
Also list any code libraries used (if applicable).

Code libraries used:
* 

### References

List the references used in the Notebook here.

E.g.

[[1]](https://doi.org/10.1038/s41598-018-20628-2) Rodriguez, D., De Voil, P., Hudson, D., Brown, J. N., Hayman, P., Marrou, H., & Meinke, H. (2018). Predicting optimum crop designs using crop models and seasonal climate forecasts. Scientific reports, 8(1), 2231.